Parts of prompts and source codes are provided

1. Text Classification

1.1 Zero-Shot Learning with GPT Embeddings

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import tiktoken
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity, get_embedding
from sklearn.metrics import PrecisionRecallDisplay

test=pd.read_csv('your file.csv')
test.columns =['prompt','completion']
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  
max_tokens = 8000  
top_n = 1000

test = test[test.n_tokens <= max_tokens]
test["embedding"] = test.prompt.apply(lambda x: get_embedding(x, engine=embedding_model))
encoding = tiktoken.get_encoding(embedding_encoding)
test["n_tokens"] = test.prompt.apply(lambda x: len(encoding.encode(x)))
EMBEDDING_MODEL = "text-embedding-ada-002"
test["embedding"] = test.embedding.apply(np.array)


def evaluate_embeddings_approach(
    labels = ['battery', 'non-battery'], 
    model = EMBEDDING_MODEL,
):
    label_embeddings = [get_embedding(label, engine=model) for label in labels]

    def label_score(review_embedding, label_embeddings):
        return cosine_similarity(review_embedding, label_embeddings[1]) - cosine_similarity(review_embedding, label_embeddings[0])

    probas = test["embedding"].apply(lambda x: label_score(x, label_embeddings))
    preds = probas.apply(lambda x: 'non-battery' if x>0 else 'battery')
    print(confusion_matrix(test.completion, preds))
    report = classification_report(test.completion, preds, digits=3)

    print(report)

    return probas, preds
    
    

evaluate_embeddings_approach(labels=['battery', 'non-battery'], model=EMBEDDING_MODEL)

1.2 Few-Shot Learning with GPT-3.5

In [ ]:
prompt  = """
The task is to classify whether a given document is TRUE or FALSE. TRUE is 'papers related to battery energy materials' and False is 'medical and psychological research'.

prompt: Anionic Methacrylate Copolymer (AMC) is a fully polymerized copolymer used in the pharmaceutical industry as an enteric/delayed-release coating to permit the pH-dependent release of active ingredients in the gastrointestinal tract from oral dosage forms. This function is of potential use for food supplements. Oral administration of radiolabeled copolymer to rats resulted in the detection of chemically unchanged copolymer in the feces, with negligible absorption (<0.1%). AMC is therefore determined not to be bioavailable. Within a genotoxicity test battery AMC did not show any evidence of genotoxicity in bacteria and mammalian cells. Furthermore, no genotoxic effects occurred in vivo within a micronucleus test. There would therefore appear to be no safety concerns under intended conditions of oral use for the discussed toxicological endpoints.
completion: False

prompt: Allergic contact dermatitis is a common skin disease and is elicited by repeated skin contact with an allergen. In the regulatory context, currently only data from animal experiments are acceptable to assess the skin sensitizing potential of substances. Animal welfare and EU Cosmetic Directive/Regulation call for the implementation of animal-free alternatives for safety assessments. The mechanisms that trigger skin sensitization are complex and various steps are involved. Therefore, a single in vitro method may not be able to accurately assess this endpoint. Non-animal methods are being developed and validated and can be used for testing strategies that ensure a reliable prediction of skin sensitization potentials. In this study, the predictivities of four in vitro assays, one in chemico and one in silico method addressing three different steps in the development of skin sensitization were assessed using 54 test substances of known sensitizing potential. The predictivity of single tests and combinations of these assays were compared. These data were used to develop an in vitro testing scheme and prediction model for the detection of skin sensitizers based on protein reactivity, activation of the Keap-1/Nrf2 signaling pathway and dendritic cell activation.
completion: False

prompt: Patients with left hemisphere (LH) or right hemisphere (RH) brain injury due to stroke were tested on a speeded, color discrimination task in which two factors were manipulated: (1) the categorical relationship between the target and the distracters and (2) the visual field in which the target was presented. Similar to controls, the RH patients were faster in detecting targets in the right visual field when the target and distracters had different color names compared to when their names were the same. This effect was absent in the LH patients, consistent with the hypothesis that injury to the left hemisphere handicaps the automatic activation of lexical codes. Moreover, the LH patients showed a reversed effect, such that the advantage of different target??distracter names was now evident for targets in the left visual field. This reversal may suggest a reorganization of the color lexicon in the right hemisphere following left hemisphere brain injury and/or the unmasking of a heightened right hemisphere sensitivity to color categories.
completion: False

prompt: In this paper, we describe the evaluation of an automated in vitro micronucleus assay using CHO-K1 cells in 96-well plates. CHO-K1 cells were pre-loaded with a cell dye that stains the cytoplasm, after which the cells were treated with the test compounds for either 3h (for the +S9 condition) or 24h (for the ???S9 condition). A total of 10 concentrations were tested, of which the top five concentrations were scored (limited by either cytotoxicity or solubility). At the end of the incubation period the cells were fixed and their DNA was stained with Hoechst. The visualization and scoring of the cells was done using an automated fluorescent microscope coupled with proprietary automated image analysis software provided by Cellomics (Pittsburg, PA). A total of 46 compounds were used in this evaluation, including 8 aneugens and 25 clastogens with varied mechanisms of action. Thirteen non-genotoxic compounds were also included. The automated scoring had a sensitivity of 88% and a specificity of 100%, with a predictive value positive of 100% and a predictive value negative of 76%, compared to data from the literature that was obtained with manual scoring. We also describe the incorporation of a metabolic activation system using rat liver S9 homogenates, and the use of cell number counts as a cytotoxicity index which is complementary to the CBPI- (cytokinesis-block proliferation index) based index. Finally, we also discuss the potential for artefactual findings due to fluorescent precipitate, which should be carefully monitored to prevent false positive results. In conclusion, the automated in vitro micronucleus scoring is a valid alternative to the manual scoring of slides, and it has the advantage of generating data in a rapid and consistent manner, and with low compound requirements, which makes it well suited as a screening assay in the early stages of compound development.
completion: False

prompt: Lithium oxide (Li2O) is activated in the presence of a layered composite cathode material (HEM) significantly increasing the energy density of lithium-ion batteries. The degree of activation depends on the current rate, electrolyte salt, and anode type. In full-cell tests, the Li2O was used as a lithium source to counter the first-cycle irreversibility of high-capacity composite alloy anodes. When Li2O is mixed with HEM to serve as a cathode, the electrochemical performance was improved in a full cell having an SiO-SnCoC composite as an anode. The mechanism behind the Li2O activation could also explain the first charge plateau and the abnormal high capacity associated with these high energy cathode materials.
completion: True

prompt: Hydrogen is a major industrial chemical whose manufacture is responsible for ???3% of global carbon dioxide emissions. >95% of hydrogen is made via reforming fossil fuels which typically co-produces hydrogen and waste carbon dioxide. Nearly all other hydrogen is co-produced with other commodity chemicals. Unfortunately, many alternative, clean hydrogen production processes are small-scale because they require major reductions in capital cost or energy prices to be economical enough for industry. Because the climate problem is urgent, and the economics of future energy is uncertain, this paper seeks to expand the options for producing industrial-scale, clean hydrogen under common, present-day economic conditions. First, we build a model to understand the economic and carbon dioxide emissions constraints of sulfur electrolysis which is an emerging process that cogenerates hydrogen and co-salable sulfuric acid and has the potential to produce up to 36% of the world's current hydrogen demand under present-day, average US economic conditions. We also use our model to evaluate water electrolysis, which cogenerates hydrogen and waste oxygen, but is not economical under present-day average US economic conditions. We then propose criteria for identifying clean hydrogen production chemistries. Using these criteria, we find enough reactions to have the combined potential to make over 150% of the world's industrial hydrogen needs under present-day, average US economic conditions while reducing cost and reducing or eliminating CO2 emissions. Given the urgency of the climate problem, we believe that an economic analysis, such as this is crucial to near-term CO2 emissions reductions.
completion: True

prompt: The reduction of oxygen was studied in 0.1M KCl at 70??C using the rotating disk electrode (RDE) technique on platinum and electrodeposited ZnO thin film electrodes deposited on platinum substrates. In the absence of Zn2+ ions in solution, a Tafel slope of 139mVdec???1 was obtained, a value close to that measured on bare platinum electrode (133mVdec???1) and ascribed to the limitation of the reaction rate by the first electron transfer. The main difference between the noble metal and the oxide electrode was a shift of the curves towards more negative potentials. In the presence of Zn2+ ions, the current density decreased significantly and the Tafel slope was measured at 282mVdec???1 showing that the electrode was partially blocked by zinc oxide formation reaction intermediates.
completion: True

prompt: Lithium non-stoichiometric Li[Li x (Ni1/3Co1/3Mn1/3)1???x ]O2 materials (0??? x ???0.17) were synthesized using a spray drying method. The electrochemical properties and structural stabilities of the synthesized materials were investigated. The synthesized materials exhibited a hexagonal structure in all the x-value and the lattice parameters of the materials were gradually decreased with increasing x-value due to an increasing amount of Ni3+ ions for charge compensation. The capacity retention ability and rate capability of the stoichiometric Li(Ni1/3Co1/3Mn1/3)O2 material were improved by increasing x-value, the so-called overlithiation. We found that the overlithiated materials could keep more structural integrity than the stoichiometric one during electrochemical cyclings, which could be one of reasons for a better electrochemical properties of the overlithiated materials.
completion: True

prompt: In the eastern province of Zambia, three companies for solar energy services have been operating for more than two years, with 400 clients paying for the use of solar photovoltaic (PV) installations. Clients do not become owners of the systems. Instead, the company continues to charge a fee for keeping the systems in operation. In this way, the useful lifetime of the solar systems is increased, and clients have local access to skilled service and spare parts. The fees for solar services cover the full operational costs of the companies, including battery replacement and makes them independent of further support from government or donors. Results to date show that clients are happy with their systems, which are of a standard solar home (SHS) size of 50 Wp, with four lights and a socket for radio or other light DC appliance. This is in spite of paying a higher price than what was previously spent on candles and kerosene for lighting. The most important benefit cited among the clients is that children now have the possibility to do their homework in the evening. The second most highly valued feature is entertainment, such as radio and video. Solar systems contribute to increased income earning opportunities especially in the service sector. The poorest rural people cannot afford to pay for a solar system in their house, but benefit from systems in the shops.
completion: True

prompt: Rechargeable Li-based electrochemical cells are regarded as the most promising candidate for electric vehicle (EV) applications. High-capacity cathode combined with Li anode is a excellent and realizable choice to promote the endurance of rechargeable Li batteries. In this paper, single phase of zirconium pyrovanadate ZrV2O7 was first synthesized via sol-gel method in a low sintering temperature of 500??C. The Li-absent mixed-metal-oxide ZrV2O7, previously regarded as negative thermal expansion (NTE) material only, revealed a high specific capacity of 282.6mAhg???1 and a voltage plateau of 2.48V vs. Li as potential cathode for rechargeable Li batteries. These findings suggest that the ZrV2O7 after further optimization can be employed as high-capacity electrode for rechargeable Li batteries.
completion: True

prompt: Laparoscopic sleeve gastrectomy (LSG) is an effective bariatric procedure, and it can be done as an isolated LSG or in conjunction with biliopancreatic diversion bypass/duodenal switch (laparoscopic duodenal switch; LDS). Gastroesophageal reflux after LSG has been described, but the mechanism is unknown and the treatment in the severest cases has not been discussed. We describe a cohort of patients who have underwent an LSG or LDS, and have suffered from a severe postoperative gastroesophageal motility disorder and/or reflux, report on their treatment, and discuss possible underlying mechanisms.
completion: 

"""

In [ ]:
import openai
result= openai.Completion.create(model='text-davinci-003', 
        prompt=prompt, 
        temperature=0,
        max_tokens=2, logprobs=2
        )
top_logprobs = result["choices"][0]["logprobs"]["top_logprobs"][0]
token_probs = {
    token: np.exp(logp) 
for token, logp in top_logprobs.items()
}
result['choices'][0]['text'] 
#str(token_probs) 
#max(list(token_probs.values()))

1.3 Few-Shot Learning with GPT-4

(1) Prompt example

In [ ]:
prompt  = """
[
{'role': 'system', 'content': "The task is to classify whether a given document is TRUE or FALSE. TRUE is 'papers related to battery energy materials' and FALSE is 'medical and psychological research'"}, 

{'role': 'user', 'content': 'In cultured A6 monolayers from distal Xenopus kidney, external Ni2+ stimulated active Na+ uptake via the epithelial Na+ channel, ENaC. Transepithelial capacitance measurements ruled out exocytosis of ENaC-containing vesicles underlying the Ni2+ effect. Na+ current noise analysis was performed using the neutral Na+-channel blocker 6-chloro-3,5-diamino-pyrazine-2-carboxamide (CDPC) and amiloride. The analysis of CDPC-induced noise in terms of a three-state channel model revealed that Ni2+ elicits an increase in the number of open channels as well as in the spontaneous open probability. While Ni2+ had no influence on CDPC-blocker kinetics, the macroscopic and microscopic blocking kinetics of amiloride were affected. Ni2+ turned out to compete with amiloride for a putative binding site but not with CDPC. Moreover, external Na+?\x80?known to compete with amiloride and so producing the ?\x80?self-inhibition?\x80? phenomenon?\x80?and Ni2+ exerted mutually exclusive analogous effects on amiloride kinetics. Na+ current kinetics revealed that Ni2+ prevents ENaC to be downregulated by self-inhibition. Co2+ behaved similarly to Ni2+, whereas Zn2+ did not. Attempts to disclose the chemical nature of the site reacting with Ni2+ suggested cysteine but not histidine as reaction partner.'}, 
{'role': 'assistant', 'content': 'False'}, 

{'role': 'user', 'content': 'Abamectin, widely used as a veterinary anthelmintic, medicine against a variety of animal parasites and insects, can runoff from the sites of application and becomes an aquatic pollutant. The aim of this study was to identify the toxicity of abamectin on bacteria, algae, daphnids, and fish. An extremely high toxicity of avermectin to the survival and reproduction of Daphnia magna was observed in 21-day exposure tests. Zebrafish and the algae Scenedesmus subspicatus are less sensitive to avermectin. The compound is expected to have adverse effects on the aquatic environment due to its high toxicity, even at very low concentrations, to daphnids and to fish.'}, {'role': 'assistant', 'content': 'False'}, {'role': 'user', 'content': 'We assessed potential toxicity of fungicides to natural bacterial communities from a constructed wetland, located in southern Sweden, and compared the sensitivity of two endpoints indicating bacterial activity, leucine incorporation, and potential denitrification, in detecting toxicity. The effects of eight fungicides (benomyl, carbendazim, carboxin, captan, cycloheximide, fenpropimorph, propiconazole, and thiram), two bactericides (bronopol and chlortetracycline) as controls, and one reference compound (3,5-dichlorophenol), were tested in a water-sediment microcosm set-up. Leucine incorporation was measured in both the water and sediment column, while potential denitrification was measured for the entire microcosm. The bactericides and the reference compound gave sigmoid concentration?\x80?response curves for both endpoints in all but one case. The fungicides thiram, captan, and benomyl, and to a lesser extent fenpropimorph and propiconazole had quantifiable toxic effects on leucine incorporation, with EC50 values ranging from 3 to 70??mg??l???1, while carbendazim, carboxin, and cycloheximide had little effect at the investigated concentrations. Only thiram and captan inhibited potential denitrification; the other fungicides showed no quantifiable effect. A greater toxic effect on leucine incorporation was recorded for bacterial communities associated with the water column, compared to the sediment column, for all tested compounds. Leucine incorporation was the more sensitive method for toxicity assessment of bacterial communities, and also allowed for a rapid and simple way of comparing exposure in the sediment and water column, making it an attractive standard method for community based toxicological assays in aquatic environments.'}, 
{'role': 'assistant', 'content': 'False'}, 

{'role': 'user', 'content': 'Diabetes is a debilitating illness requiring lifelong management. Treatments can be varied in terms of mode of administration as well as type of agent. Unfortunately, most patient reported outcome measures currently available to assess the impact of treatment are specific to diabetes type, treatment modality or delivery systems and are designed to be either a HRQoL or treatment satisfaction measure. To address these gaps, the Treatment Related Impact Measure-Diabetes and Device measures were developed. This paper presents the item development and validation of the TRIM Diabetes/Device.'}, 
{'role': 'assistant', 'content': 'False'}, 

{'role': 'user', 'content': 'Palladium (Pd) incorporated poly (3,4-ethylenedioxythiophene) (PEDOT) films were synthesized through an electrochemical route and characterized using scanning electron microscopy (SEM) and atomic force microscopy (AFM). The electrochemical study showed catalytic oxidation of dopamine (DA) with optimum loading of Pd. DA and uric acid (UA) were detected using differential pulse voltammetry (DPV). In the presence of ascorbic acid (AA), DA-AA showed peak potential separation of 0.19??V while 0.32??V between UA-AA on Pd-incorporated PEDOT. These peak separations are large enough for sensing DA and UA in the presence of AA. DA and UA exhibited linear calibration plots and the minimum detection limits are 0.5 and 7????M respectively. On Pd-PEDOT, the reversibility of DA oxidation was found to increase compared to bare glassy carbon electrode (GCE) and PEDOT modified GCE. Fouling effects were also found to be minimal making Pd-PEDOT composite suitable for electroanalysis.'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'The use of oil in road transport contributes significantly to global greenhouse gas (GHG) emissions in addition to rapidly depleting this non-renewable resource. Emissions can be greatly reduced by the total replacement of petroleum-based vehicles with electric vehicles using a combination of hydrogen and battery energy storage technologies. This paper analytically reviews the potential reduction in Australian road transport GHG emissions through the total replacement of petroleum-fueled vehicles with hydrogen and battery electrical vehicles by 2050. If electricity for hydrogen production & storage and battery charging is sourced from the national electricity grid, it is estimated that emissions can be reduced by between 56% and 73% in 2050 compared to the Australian Government?\x80?s Bureau of Infrastructure, Transport and Regional Economics (BITRE) projections, depending on the current range of government carbon price projections. Emissions can be reduced even further by supplementing grid electricity with standalone renewable electricity dedicated to hydrogen production and storage. It is found that there is more than sufficient renewable energy resources within Australia (particularly solar and wind) to meet the significant increase in the annual electricity generation that would be required to implement this strategy.'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'Long-term electrochemical behaviors of hydrated MnO2 electrochemical capacitor (EC) electrode in alkali chloride (KCl(aq)) electrolyte have been studied by using potential cycling for thousands of cycles within different potential windows spanning from 0.8V (versus Ag/AgCl(aq)) to varied lower-end potentials below the open-circuit potential. Three potential ranges resulting in different cycling behaviors of the oxide EC have been identified. Range I: cycling above 0.2V results in no change in either microstructure or surface chemistry of the oxide electrode, and no capacitance reduction has been observed. Range II: cycling down to 0.0V leads to extensive morphological reconstruction and limited reduction of surface Mn ions, while the electrode capacitance remains stable. Range III: cycling with lower-potential end below 0.0V results in obvious capacitance reduction, along with different morphological reconstruction and Mn reduction from those in Range II. For each selected lower-end potential in Range III, the capacitance descends to a plateau within first thousand cycles, and the extent of the capacitance reduction increases as the lower-end potential decreases.'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'We propose a novel surfactant-mediated process to fabricate flexible microsupercapacitors (MSCs) combining conventional photolithography and electrochemical deposition. The anionic surfactant mediates the process of electropolymerisation at a lower anodic potential while causing template effects in producing porous conducting poly(3,4-ethylenedioxythiophene) (PEDOT) electrodes. Using this strategy, PEDOT MSCs with remarkable performance in terms of tunable frequency response and energy density are achieved. Specifically, ultrahigh scan rate capability up to 500V/s is achieved with a crossover frequency of 400Hz at a phase angle of ???45??. This is the first polymer-based redox microsupercapacitor with excellent frequency characteristics other than carbonaceous-based electrochemical double layer capacitors reported so far in the literature. Thus, the micro-supercapacitors exhibit maximum areal cell capacitance of 9mF/cm2 with a volumetric stack capacitance of 50F/cm3 in 1M H2SO4 aqueous electrolyte. The flexibility and stability of these PEDOT MSCs is tested in aqueous gel electrolyte which showed a capacitance retention up to 80% over 10,000 cycles with a Coulombic efficiency of 100%. The maximum energy density of solid state ion gel based PEDOT MSCs was found to be 7.7mWh/cm3, which is comparable to the lithium based thin film batteries and superior to the current state-of-the-art carbon and metal oxide based MSCs. Further, the tandem configuration of flexible solid state ion gel based PEDOT MSCs is employed to demonstrate it as a power source for glowing a red light emitting diode.'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'Hybrid films composed of poly(luminol) and nanometer-sized clusters of polyoxometalate, SiMo12O40 4??? and PMo12O40 3??? have been prepared in acidic aqueous solutions. These films are stable and electrochemically active, and produced on glassy carbon, platinum, gold and transparent semiconductor tin oxide electrodes. The electrochemical quartz crystal microbalance and cyclic voltammetry were used to study in situ growth of the hybrid poly(luminol)/SiMo12O40 4??? and poly(luminol)/PMo12O40 3???. Both the poly(luminol)/SiMo12O40 4??? and poly(luminol)/PMo12O40 3??? hybrid films showed four redox couples and the electrochemical properties were compared to SiMo12O40 4??? and PMo12O40 3???. When transferred to various acidity aqueous solutions, the four redox couples and the formal potentials of two hybride film were observed to be pH-dependent. The electrocatalytic reduction of ClO3 ???, BrO3 ???, IO3 ???, S2O8 2??? and NO2 ??? by a poly(luminol)/PMo12O40 3??? hybrid film in an acidic aqueous solution showed an electrocatalytic reduction activity of IO3 ??? >BrO3 ??? and ClO3 ???. The electrocatalytic oxidation of dopamine and epinephrine by a poly(luminol)/PMo12O40 3??? hybrid film was also investigated.'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'A LiCoO2 thin film with a crystalline structure and free of the cubic phase is obtained by adopting a two-step treatment with a combination of bias substrate during RF sputtering and a post-annealing treatment at moderate temperature of 500??C or 400??C. All-solid-state Li/LiPON/LiCoO2 lithium batteries are fabricated using such crystalline bias LiCoO2 thin films. The crack-free surface of the cathode thin film and the nanosized LiCoO2 particles ensure an efficient electrochemical behavior and high performance of the battery. The rate capability and the influence of the voltage range on the rechargeable capacity delivered by the battery are investigated as well as cycling properties. A high capacity close to 50??Ahcm???2 ??m???1 is demonstrated at 10??A.cm???2 with an excellent rate capability since 86% of this value is recovered at 30C without any further damage of the cathode thin film. An excellent cycling behavior of the biased film is demonstrated since after a capacity decay of 12% over the first 100 cycles, the specific capacity practically does not decline over 800 additional cycles at 0.4C rate. These results compare very well with the best all-solid-state Li microbatteries built with LiCoO2 heat-treated at 700??C or more. It exemplifies the relevance of using biased LiCoO2 thin films heat-treated at moderate temperature (400??C-500??C).'}, 
{'role': 'assistant', 'content': 'True'}, 

{'role': 'user', 'content': 'Although photographic assessment has been found to be reliable in assessing hair loss in Trichotillomania, the validity of this method is unclear, particularly for gauging progress in treatment. The current study evaluated the psychometric properties of photographic assessment of change in Trichotillomania. Photographs showing hair loss of adults with Trichotillomania were taken before and after participating in a clinical trial for the condition. Undergraduate college students (N?\x80?=?\x80?211) rated treatment response according to the photos, and additional archival data on hair pulling severity and psychosocial health were retrieved from the clinical trial. Photographic assessment of change was found to possess fair reliability (ICC?\x80?=?\x80?0.53), acceptable criterion validity (r?\x80?=?\x80?0.51), good concurrent validity (r?\x80?=?\x80?0.30?\x80?0.36), and excellent incremental validity (??R2?\x80?=?\x80?8.67, p?\x80?<?\x80?0.01). In addition, photographic measures were significantly correlated with change in quality of life (r?\x80?=?\x80?0.42), and thus could be considered an index of the social validity of Trichotillomania treatment. Gender of the photo rater and pulling topography affected the criterion validity of photographic assessment (partial ??2?\x80?=?\x80?0.05?\x80?0.11). Recommendations for improving photographic assessment and future directions for hair pulling research are discussed.'}]

"""

(2) GPT-4 usage, where chat is default

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='your key')
result= client.chat.completions.create(model='gpt-4-0613',
        messages=prompt, 
        temperature=0,
        max_tokens=2,         
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
pred= result.choices[0].message

1.4 Fine-tuning of GPT-3

(1) data processing

In [2]:
train_df = pd.read_json("your file.jsonl")
train_df.head()

                                              prompt completion
0  The surface energy balance includes a term for...      False
1  In an attempt to forge tighter social relation...      False
2  In recent years, the retail market industry ha...      False
3  Laser physical vapor deposition is used to dep...       True
4  One of the limitations of power generation wit...       True


In [ ]:
!openai tools fine_tunes.prepare_data -f /dataset/train_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/val_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/test_df.jsonl -q

(2) fine-tuning

In [ ]:
!openai api fine_tunes.create -t "train.jsonl" -v "valid.jsonl" -m "davinci" --suffix f"YOUR ALIAS" --batch_size 1 --n_epochs 4 --learning_rate_multiplier 0.01 --prompt_loss_weight 0.01

(3) use of fine-tuned model

In [ ]:
openai.organization =""
openai.api_key =""
your_model = ""
prompt = "Reduced graphene oxide \/ ZnO nanohybrids : Metallic Zn powder induced one - step synthesis for enhanced photocurrent and photocatalytic response Reduced graphene oxide hybridized hierarchical ZnO nanorods ( RGO \/ ZnO ) were fabricated through thermal treatment of aqueous solution containing metallic Zn powder , Zn(NO3)2·6H2O , graphene oxide ( GO ) , and NaOH at 110 ° C . This one - spot , additives - free method successfully made metallic Zn powder a reducing agent of GO , a precursor of ZnO , and also a morphology controller of RGO \/ ZnO . RGO \/ ZnO nanohybrids with 4wt- % of RGO displayed optimal photocurrent and photocatalytic response under UV irradiation with 10 times and 2.2 times that of pure ZnO nanoflowers , respectively . Strong coupling and electronic interaction between GO and ZnO were verified by using XPS measurement and photoelectrochemical technique . The combination of supreme absorption ( of UV light and dye ) , suppressed photogenerated carriers recombination , and decreased solid interlayer resistance of nanohybrids contributed to their superior photochemical properties .  ==>\\n"
result = openai.Completion.create(model=your_model,        
            prompt=prompt,
            temperature=0,
            max_tokens=256,
            best_of=1,
            top_p=1,
            stop=["\n\n###\n\n"]
            )
print(result)

2. Named Entity Recognition

2.1 Few-Shot Learning with Random Retreival (GPT-3.5)

(1) Prompt example

In [ ]:
prompt = """
prompt: The prepared chitosan - capped gold nanoparticles possess all the requirements to be engulfed by the cells without causing any morphological or functional disruption : the particles are spherical , have a size of about 50 nm and are positively charged due to the quaternary amines present in the polymeric structure . Moreover , the functional groups presented on their surface can contribute to further nanoparticle conjugation with various molecules such as insulin or cisplatin for desired applications such as nanocarriers or drug delivery [ 41,42 ] . The comparative viability tests presented in this study come in agreement with other reported observations that gold nanoparticles citrate- or CTAB - coating has a dramatic impact on cell viability , development and functionality [ 43,44 ] .  ==>\n\n 

completion:  The prepared chitosan - capped gold nanoparticles possess all the requirements to be engulfed by the cells without causing any morphological or functional disruption : the particles are @@spherical## , have a size of about 50 nm and are positively charged due to the quaternary amines present in the polymeric structure . Moreover , the functional groups presented on their surface can contribute to further nanoparticle conjugation with various molecules such as insulin or cisplatin for desired applications such as nanocarriers or drug delivery [ 41,42 ] . The comparative viability tests presented in this study come in agreement with other reported observations that gold nanoparticles citrate- or CTAB - coating has a dramatic impact on cell viability , development and functionality [ 43,44 ] .  \n\n###\n\n

prompt: The optical properties of the gold seeds and the high aspect ratio nanorods were characterized by UV−vis absorption spectroscopy . Figure 2b shows a UV−vis absorption spectrum of the gold seeds . A weak band centered at ∼480 nm was observed . The particle size of the gold seeds , as determined by the high - resolution TEM images , is about 1−2 nm ( see Supporting Information ) . Previously , the gold seeds prepared using trisodium citrate as the capping agent have diameters of 2−4 nm and showed a single surface plasmon resonance ( SPR ) band at 508 nm . In general , spherical gold nanoparticles with a uniform and smaller diameter exhibit an absorption band at shorter wavelengths . The UV−vis absorption spectra of the high aspect ratio gold nanorods are shown in Figure 2c , d . The spectra show an absorption band at 497 nm , which should correspond to the transverse SPR absorption band of the nanorods . The longitudinal SPR band appears at ∼2135 nm . The position of the longitudinal SPR band is in fair agreement with that predicted using the discrete dipole approximation 834 method ( that is , lmax & equals ; 96AR & plus ; 418 , where AR refers to aspect ratio ) , considering the length distribution of the nanorods and the relatively broad width of the band . Two more bands centered at ∼830 and 1030 nm are attributed to light absorption by the triangular and truncated triangular nanoplates ( ∼90−150 nm in width ) . Since much fewer nanoplates were formed , the absorbance of the nanoplate bands was substantially lowered such that the nanoplate absorbance is similar to that of the transverse SPR band of the nanorods . Previously , the absorbance of the nanoplates was much stronger than that of the nanorods.11 This is strong evidence of the significant reduction in the formation of triangular nanoplates . Please note that a direct comparison of the relative absorbance values to determine the absolute percentages of different gold nanostructures is not accurate , as they may have different extinction coefficients .  ==>\n\n 

completion:  The optical properties of the gold seeds and the high aspect ratio nanorods were characterized by UV−vis absorption spectroscopy . Figure 2b shows a UV−vis absorption spectrum of the gold seeds . A weak band centered at ∼480 nm was observed . The particle size of the gold seeds , as determined by the high - resolution TEM images , is about 1−2 nm ( see Supporting Information ) . Previously , the gold seeds prepared using trisodium citrate as the capping agent have diameters of 2−4 nm and showed a single surface plasmon resonance ( SPR ) band at 508 nm . In general , @@spherical## gold nanoparticles with a uniform and smaller diameter exhibit an absorption band at shorter wavelengths . The UV−vis absorption spectra of the high aspect ratio gold nanorods are shown in Figure 2c , d . The spectra show an absorption band at 497 nm , which should correspond to the transverse SPR absorption band of the nanorods . The longitudinal SPR band appears at ∼2135 nm . The position of the longitudinal SPR band is in fair agreement with that predicted using the discrete dipole approximation 834 method ( that is , lmax & equals ; 96AR & plus ; 418 , where AR refers to aspect ratio ) , considering the length distribution of the nanorods and the relatively broad width of the band . Two more bands centered at ∼830 and 1030 nm are attributed to light absorption by the @@triangular## and @@truncated triangular## nanoplates ( ∼90−150 nm in width ) . Since much fewer nanoplates were formed , the absorbance of the nanoplate bands was substantially lowered such that the nanoplate absorbance is similar to that of the transverse SPR band of the nanorods . Previously , the absorbance of the nanoplates was much stronger than that of the nanorods.11 This is strong evidence of the significant reduction in the formation of @@triangular## nanoplates . Please note that a direct comparison of the relative absorbance values to determine the absolute percentages of different gold nanostructures is not accurate , as they may have different extinction coefficients .  \n\n###\n\n

prompt: From the TEM image of Au@PS core – shell particle ( Fig . 6 ) , it is evident that the size of spherical Au particle is ~115 nm , and the outside PS shell ( formed by emulsifier - free polymerization ) has a thickness of 2–3 nm .  ==>\n\n 

completion:  From the TEM image of Au@PS core – shell particle ( Fig . 6 ) , it is evident that the size of @@spherical## Au particle is ~115 nm , and the outside PS shell ( formed by emulsifier - free polymerization ) has a thickness of 2–3 nm .  \n\n###\n\n

prompt: TEM image of Au nanoparticles was shown in Fig . 4a with an average size of about 20 nm , it can observed that the photochemical method to prepare Au nanoparticles was successful . SEM images of ZnO nanorods films and AuNPs / ZnO nano - composite films were shown in Fig . 4b and Fig . 4c . Fig . 4b shows the uniformly hexagonal shapes of ZnO nanorods with an average diameter of about 50 nm and Fig . 4c shows Au nanoparticles was attached on ZnO nanorods , and the self - assembly method for preparing Au nanoparticles on ZnO nanorods was successful .  ==>\n

completion:
"""

(2) GPT-3.5 model usage, where the maximum number of tokens should be determined with tiktoken

In [ ]:
import openai
num_tokens = 256
result= openai.Completion.create(model='text-davinci-003', 
        prompt=prompt, 
        temperature=0,
        max_tokens=num_tokens, logprobs=2
        )
top_logprobs = result["choices"][0]["logprobs"]["top_logprobs"][0]
token_probs = {
    token: np.exp(logp) 
for token, logp in top_logprobs.items()
}
result['choices'][0]['text'] 

2.2 Few-Shot Learning with Task-Informed Random Retreival (GPT-3.5)

(1) Prompt example

In [ ]:
prompt = """
The task is to extract the descriptive entities in the given documents. Below are some examples.

prompt: The prepared chitosan - capped gold nanoparticles possess all the requirements to be engulfed by the cells without causing any morphological or functional disruption : the particles are spherical , have a size of about 50 nm and are positively charged due to the quaternary amines present in the polymeric structure . Moreover , the functional groups presented on their surface can contribute to further nanoparticle conjugation with various molecules such as insulin or cisplatin for desired applications such as nanocarriers or drug delivery [ 41,42 ] . The comparative viability tests presented in this study come in agreement with other reported observations that gold nanoparticles citrate- or CTAB - coating has a dramatic impact on cell viability , development and functionality [ 43,44 ] .  ==>\n\n 

completion:  The prepared chitosan - capped gold nanoparticles possess all the requirements to be engulfed by the cells without causing any morphological or functional disruption : the particles are @@spherical## , have a size of about 50 nm and are positively charged due to the quaternary amines present in the polymeric structure . Moreover , the functional groups presented on their surface can contribute to further nanoparticle conjugation with various molecules such as insulin or cisplatin for desired applications such as nanocarriers or drug delivery [ 41,42 ] . The comparative viability tests presented in this study come in agreement with other reported observations that gold nanoparticles citrate- or CTAB - coating has a dramatic impact on cell viability , development and functionality [ 43,44 ] .  \n\n###\n\n

prompt: The optical properties of the gold seeds and the high aspect ratio nanorods were characterized by UV−vis absorption spectroscopy . Figure 2b shows a UV−vis absorption spectrum of the gold seeds . A weak band centered at ∼480 nm was observed . The particle size of the gold seeds , as determined by the high - resolution TEM images , is about 1−2 nm ( see Supporting Information ) . Previously , the gold seeds prepared using trisodium citrate as the capping agent have diameters of 2−4 nm and showed a single surface plasmon resonance ( SPR ) band at 508 nm . In general , spherical gold nanoparticles with a uniform and smaller diameter exhibit an absorption band at shorter wavelengths . The UV−vis absorption spectra of the high aspect ratio gold nanorods are shown in Figure 2c , d . The spectra show an absorption band at 497 nm , which should correspond to the transverse SPR absorption band of the nanorods . The longitudinal SPR band appears at ∼2135 nm . The position of the longitudinal SPR band is in fair agreement with that predicted using the discrete dipole approximation 834 method ( that is , lmax & equals ; 96AR & plus ; 418 , where AR refers to aspect ratio ) , considering the length distribution of the nanorods and the relatively broad width of the band . Two more bands centered at ∼830 and 1030 nm are attributed to light absorption by the triangular and truncated triangular nanoplates ( ∼90−150 nm in width ) . Since much fewer nanoplates were formed , the absorbance of the nanoplate bands was substantially lowered such that the nanoplate absorbance is similar to that of the transverse SPR band of the nanorods . Previously , the absorbance of the nanoplates was much stronger than that of the nanorods.11 This is strong evidence of the significant reduction in the formation of triangular nanoplates . Please note that a direct comparison of the relative absorbance values to determine the absolute percentages of different gold nanostructures is not accurate , as they may have different extinction coefficients .  ==>\n\n 

completion:  The optical properties of the gold seeds and the high aspect ratio nanorods were characterized by UV−vis absorption spectroscopy . Figure 2b shows a UV−vis absorption spectrum of the gold seeds . A weak band centered at ∼480 nm was observed . The particle size of the gold seeds , as determined by the high - resolution TEM images , is about 1−2 nm ( see Supporting Information ) . Previously , the gold seeds prepared using trisodium citrate as the capping agent have diameters of 2−4 nm and showed a single surface plasmon resonance ( SPR ) band at 508 nm . In general , @@spherical## gold nanoparticles with a uniform and smaller diameter exhibit an absorption band at shorter wavelengths . The UV−vis absorption spectra of the high aspect ratio gold nanorods are shown in Figure 2c , d . The spectra show an absorption band at 497 nm , which should correspond to the transverse SPR absorption band of the nanorods . The longitudinal SPR band appears at ∼2135 nm . The position of the longitudinal SPR band is in fair agreement with that predicted using the discrete dipole approximation 834 method ( that is , lmax & equals ; 96AR & plus ; 418 , where AR refers to aspect ratio ) , considering the length distribution of the nanorods and the relatively broad width of the band . Two more bands centered at ∼830 and 1030 nm are attributed to light absorption by the @@triangular## and @@truncated triangular## nanoplates ( ∼90−150 nm in width ) . Since much fewer nanoplates were formed , the absorbance of the nanoplate bands was substantially lowered such that the nanoplate absorbance is similar to that of the transverse SPR band of the nanorods . Previously , the absorbance of the nanoplates was much stronger than that of the nanorods.11 This is strong evidence of the significant reduction in the formation of @@triangular## nanoplates . Please note that a direct comparison of the relative absorbance values to determine the absolute percentages of different gold nanostructures is not accurate , as they may have different extinction coefficients .  \n\n###\n\n

prompt: From the TEM image of Au@PS core – shell particle ( Fig . 6 ) , it is evident that the size of spherical Au particle is ~115 nm , and the outside PS shell ( formed by emulsifier - free polymerization ) has a thickness of 2–3 nm .  ==>\n\n 

completion:  From the TEM image of Au@PS core – shell particle ( Fig . 6 ) , it is evident that the size of @@spherical## Au particle is ~115 nm , and the outside PS shell ( formed by emulsifier - free polymerization ) has a thickness of 2–3 nm .  \n\n###\n\n

prompt: TEM image of Au nanoparticles was shown in Fig . 4a with an average size of about 20 nm , it can observed that the photochemical method to prepare Au nanoparticles was successful . SEM images of ZnO nanorods films and AuNPs / ZnO nano - composite films were shown in Fig . 4b and Fig . 4c . Fig . 4b shows the uniformly hexagonal shapes of ZnO nanorods with an average diameter of about 50 nm and Fig . 4c shows Au nanoparticles was attached on ZnO nanorods , and the self - assembly method for preparing Au nanoparticles on ZnO nanorods was successful .  ==>\n

completion:
"""

2.3 Few-Shot Learning with kNN retreival (GPT-3.5)

(1) Prompt example

In [ ]:
prompt = """ 
The task is to extract the descriptive entities in the given documents. Below are some examples.

prompt: Self - organization of monodisperse nanoscale building blocks into ordered arrays represents a facile and versatile route for metamaterials with rationally modulated properties . In particular , liquid - crystalline - type superstructures or superlattices composed of anisotropic nanocrystals have attracted a fast - growing interest due to the rich phase behaviors resulting from the added complexity of shape anisotropy and the potential for emergent collective properties . Here we explore the intriguing shape - dependent packing symmetry of ordered NR assemblies by slow evaporation of concentrated gold NR aqueous solutions at room temperature . Onsager ’s theory on the assembly behavior of hard rodlike colloids shows that above a critical concentration of rods there is a net gain in entropy on transition from isotropic random packing to a nematic liquid crystalline phase . As shown in Figure 10 , large - area smectic phases form in dense NR solids , where NRs self - organize into close - packed layers and adjacent layers align nearly parallel into ordered superstructures . AFM studies confirm the three - dimensional nature of gold NR assemblies with a typical domain size of ∼10 μm2 ( Figures 10i , S43 ) . AFM height analysis along layers of smectic phases reveals periodic height modulation with a periodicity of 28.0 ± 2.5 nm ( the NR diameter is measured to be 22.0 ± 1.0 nm from TEM images ) , which is consistent with recent experimental studies showing partially interdigitated CTAB bilayers on the gold NR surface ( Figure 10j , k ) . Moreover , the surfaces of gold NR assemblies are not perfectly flat but exhibit height variations corresponding to one or two layers of gold NRs ( Figure 10j , k ) , which could be advantageous for surface - enhanced spectroscopy due to the increased accessibility of local hot - spots . However , because of the drying process on a solid substrate ( TEM grid in this case ) , fluctuations in the local NR concentration are expected and the extent of spatial ordering for monodisperse gold NRs is largely limited by the area over which a uniform concentration of NRs can be maintained during the very last stage of deposition . Indeed , this is reflected by the coexistence of smectic and nematic phases on a single TEM grid for gold NRs of various aspect ratios . The nematic phase , characterized by a uniaxial orientational order and lack of positional order , usually forms at less concentrated regions and is more common for NRs of larger aspect ratios ( Figures S47–S49 ) . Interestingly , for gold NRs with an aspect ratio less than 4.0 , bilayered motifs exhibiting approximately centered - rectangular local packing symmetry are routinely observed ( Figures S35 , S36 , S39 , and S45 ) , which has been rarely seen in other NR assemblies . Moreover , the nematic or smectic ordering can be disrupted through continuous bending of neighboring layers , reminiscent of nonequilibrium drying and complex interactions between the NRs ( Figure 10e , f ) . Those wavy gold nanorod assemblies have also been reported by Murphy et al . and referred to as “ swirling clustering ” or “ end - to - end cementing ” . Therefore , theories on the packing of hard rods under equilibrium conditions can not fully explain the observed structural complexity of gold NR assemblies , and development of a rigorous framework taking into account not only entropic contributions but also nanoscale forces including van der Waals attractions , electrostatic repulsions , dipole – dipole interactions , and , in particular , the role of CTAB layers and counterion interaction in self - assembling gold nanorods is needed .  ==>\n
completion:  Self - organization of monodisperse nanoscale building blocks into ordered arrays represents a facile and versatile route for metamaterials with rationally modulated properties . In particular , liquid - crystalline - type superstructures or superlattices composed of anisotropic nanocrystals have attracted a fast - growing interest due to the rich phase behaviors resulting from the added complexity of shape anisotropy and the potential for emergent collective properties . Here we explore the intriguing shape - dependent packing symmetry of ordered NR assemblies by slow evaporation of concentrated gold NR aqueous solutions at room temperature . Onsager ’s theory on the assembly behavior of hard @@rodlike## colloids shows that above a critical concentration of rods there is a net gain in entropy on transition from isotropic random packing to a nematic liquid crystalline phase . As shown in Figure 10 , large - area smectic phases form in dense NR solids , where NRs self - organize into close - packed layers and adjacent layers align nearly parallel into ordered superstructures . AFM studies confirm the three - dimensional nature of gold NR assemblies with a typical domain size of ∼10 μm2 ( Figures 10i , S43 ) . AFM height analysis along layers of smectic phases reveals periodic height modulation with a periodicity of 28.0 ± 2.5 nm ( the NR diameter is measured to be 22.0 ± 1.0 nm from TEM images ) , which is consistent with recent experimental studies showing partially interdigitated CTAB bilayers on the gold NR surface ( Figure 10j , k ) . Moreover , the surfaces of gold NR assemblies are not perfectly flat but exhibit height variations corresponding to one or two layers of gold NRs ( Figure 10j , k ) , which could be advantageous for surface - enhanced spectroscopy due to the increased accessibility of local hot - spots . However , because of the drying process on a solid substrate ( TEM grid in this case ) , fluctuations in the local NR concentration are expected and the extent of spatial ordering for monodisperse gold NRs is largely limited by the area over which a uniform concentration of NRs can be maintained during the very last stage of deposition . Indeed , this is reflected by the coexistence of smectic and nematic phases on a single TEM grid for gold NRs of various aspect ratios . The nematic phase , characterized by a uniaxial orientational order and lack of positional order , usually forms at less concentrated regions and is more common for NRs of larger aspect ratios ( Figures S47–S49 ) . Interestingly , for gold NRs with an aspect ratio less than 4.0 , bilayered motifs exhibiting approximately centered - rectangular local packing symmetry are routinely observed ( Figures S35 , S36 , S39 , and S45 ) , which has been rarely seen in other NR assemblies . Moreover , the nematic or smectic ordering can be disrupted through continuous bending of neighboring layers , reminiscent of nonequilibrium drying and complex interactions between the NRs ( Figure 10e , f ) . Those wavy gold nanorod assemblies have also been reported by Murphy et al . and referred to as “ swirling clustering ” or “ end - to - end cementing ” . Therefore , theories on the packing of hard rods under equilibrium conditions can not fully explain the observed structural complexity of gold NR assemblies , and development of a rigorous framework taking into account not only entropic contributions but also nanoscale forces including van der Waals attractions , electrostatic repulsions , dipole – dipole interactions , and , in particular , the role of CTAB layers and counterion interaction in self - assembling gold nanorods is needed .  \n\n###\n\n

prompt: In order to monitor the growth of AuNRs by TEM , aliquots were removed from the growth solution every hour , their absorbance spectra were measured , and they were incubated with MUTAB for 2 days to ensure that a maximum surface coverage had occurred . TEM images of these rods after 3 , 7 , and 11 h of growth are shown in Figure 5 ( see Figure S2 , Supporting Information , for the 5 , 9 , and 12 h points ) . Figure 6 details the change in AuNRs dimensions extracted from the TEM images . Specifically , the changes in length ( Figure 6A ) , waist and tip diameter ( Figure 6B ) , and total volume , estimated by assuming a cylindrical structure with diameter averaged between the waist and tips ( Figure 6C ) , are plotted against the growth time . A summary of this information is provided in Table S2 ( Supporting Information ) . After 3 h , AuNRs display the standard cylindrical morphology with the aspect ratio and LSPR peak close to the final values . However , at this point the rods have grown to only about 15 % of their final size based on the estimated volume . The linear increase in volume from that point shown in Figure 6A demonstrates that the reduction to metallic gold occurred at a roughly constant rate , assuming that no new nucleation events are taking place in solution . This steady growth in size is in contrast to the “ popcorn ” model of ascorbic acid - mediated nanorods growth proposed by Edgar and co - workers . This model states that rather than following the traditional assumption that all seed particles grow evenly from the start of the reaction to the end , the seed particles actually lie quiescent for variable amounts of time and then rapidly grow to their final size . This model was supported by the presence of both small and nearly fully grown AuNRs starting from the very beginning of the reaction up until nearly the end , which is clearly not evident in our procedure . However , it may be that due to the reduced growth rate , most seeds that will eventually grow have time to exit the quiescent state within the first few hours of growth while nanorods have only had time to grow ∼15 % of their final size . In other words , the rate at which seed particles switch from the quiescent to the growing state appears to be faster than the rate of AuNRs growth .  ==>\n
completion:  In order to monitor the growth of AuNRs by TEM , aliquots were removed from the growth solution every hour , their absorbance spectra were measured , and they were incubated with MUTAB for 2 days to ensure that a maximum surface coverage had occurred . TEM images of these rods after 3 , 7 , and 11 h of growth are shown in Figure 5 ( see Figure S2 , Supporting Information , for the 5 , 9 , and 12 h points ) . Figure 6 details the change in AuNRs dimensions extracted from the TEM images . Specifically , the changes in length ( Figure 6A ) , waist and tip diameter ( Figure 6B ) , and total volume , estimated by assuming a @@cylindrical## structure with diameter averaged between the waist and tips ( Figure 6C ) , are plotted against the growth time . A summary of this information is provided in Table S2 ( Supporting Information ) . After 3 h , AuNRs display the standard @@cylindrical## morphology with the aspect ratio and LSPR peak close to the final values . However , at this point the rods have grown to only about 15 % of their final size based on the estimated volume . The linear increase in volume from that point shown in Figure 6A demonstrates that the reduction to metallic gold occurred at a roughly constant rate , assuming that no new nucleation events are taking place in solution . This steady growth in size is in contrast to the “ popcorn ” model of ascorbic acid - mediated nanorods growth proposed by Edgar and co - workers . This model states that rather than following the traditional assumption that all seed particles grow evenly from the start of the reaction to the end , the seed particles actually lie quiescent for variable amounts of time and then rapidly grow to their final size . This model was supported by the presence of both small and nearly fully grown AuNRs starting from the very beginning of the reaction up until nearly the end , which is clearly not evident in our procedure . However , it may be that due to the reduced growth rate , most seeds that will eventually grow have time to exit the quiescent state within the first few hours of growth while nanorods have only had time to grow ∼15 % of their final size . In other words , the rate at which seed particles switch from the quiescent to the growing state appears to be faster than the rate of AuNRs growth .  \n\n###\n\n

prompt: The detailed morphology and component characterizations were presented with TEM images and XRD patterns . Fig . 1 ( a ) gives the TEM image of pure ZnO , and Fig . 1(b ) exhibits the initial Au NRs with an average length / diameter of 15 ± 3 nm/50 ± 3 nm . Fig . 1(c ) shows that the Au@ZnO NRs exhibit a standard core - shell nanostructure . The Au cores with special dumbbell - like morphology is ascribed to the overgrowth of Au NRs , the corresponding growth mechanism had been discussed in our previous work [ 11 ] . The ZnO nanoshells have an average thickness of 52 nm and display a loose form . As observed in Fig . 1(d ) , the annealed Au@ZnO NRs keep uniform core - shell structures , and its Au cores still maintain rod - like morphology . The ZnO nanoshells show dense morphology , suggesting that its crystallinity is highly enhanced .  ==>\n
completion:  The detailed morphology and component characterizations were presented with TEM images and XRD patterns . Fig . 1 ( a ) gives the TEM image of pure ZnO , and Fig . 1(b ) exhibits the initial Au NRs with an average length / diameter of 15 ± 3 nm/50 ± 3 nm . Fig . 1(c ) shows that the Au@ZnO NRs exhibit a standard core - shell nanostructure . The Au cores with special @@dumbbell## - @@like## morphology is ascribed to the overgrowth of Au NRs , the corresponding growth mechanism had been discussed in our previous work [ 11 ] . The ZnO nanoshells have an average thickness of 52 nm and display a loose form . As observed in Fig . 1(d ) , the annealed Au@ZnO NRs keep uniform core - shell structures , and its Au cores still maintain @@rod## - @@like## morphology . The ZnO nanoshells show dense morphology , suggesting that its crystallinity is highly enhanced .  \n\n###\n\n

prompt: Fig . 4 a corresponds to the TEM images of AuNRs dispersion in the water having length ≈53–60 nm and width ≈20–25 nm ( aspect ratio ≈ 2.5 ) along with some square or rectangular shaped particles ( SI - Fig . 2 ) . MeOH addition resulted in the decrease of inter particle distance among the adjacent NRs and showed parallel alignment to a little extent ( Fig . 4b ) . Formation of small groups of NRs with random or haphazard arrangement ( Fig . 4c ) and a large clump of aggregate ( Fig . 4d ) relative to distantly positioned rods in aqueous media are also seen in MeOH dispersion .  ==>\n
completion:
"""

2.4 Few-Shot Learning with kNN retreival (GPT-4)

In [ ]:
prompt = """ 
[{'role': 'system', 'content': 'The task is to extract the morphological entities in the given documents. Below are some examples.'}, 

{'role': 'user', 'content': 'Self - organization of monodisperse nanoscale building blocks into ordered arrays represents a facile and versatile route for metamaterials with rationally modulated properties . In particular , liquid - crystalline - type superstructures or superlattices composed of anisotropic nanocrystals have attracted a fast - growing interest due to the rich phase behaviors resulting from the added complexity of shape anisotropy and the potential for emergent collective properties . Here we explore the intriguing shape - dependent packing symmetry of ordered NR assemblies by slow evaporation of concentrated gold NR aqueous solutions at room temperature . Onsager ’s theory on the assembly behavior of hard rodlike colloids shows that above a critical concentration of rods there is a net gain in entropy on transition from isotropic random packing to a nematic liquid crystalline phase . As shown in Figure 10 , large - area smectic phases form in dense NR solids , where NRs self - organize into close - packed layers and adjacent layers align nearly parallel into ordered superstructures . AFM studies confirm the three - dimensional nature of gold NR assemblies with a typical domain size of ∼10 μm2 ( Figures 10i , S43 ) . AFM height analysis along layers of smectic phases reveals periodic height modulation with a periodicity of 28.0 ± 2.5 nm ( the NR diameter is measured to be 22.0 ± 1.0 nm from TEM images ) , which is consistent with recent experimental studies showing partially interdigitated CTAB bilayers on the gold NR surface ( Figure 10j , k ) . Moreover , the surfaces of gold NR assemblies are not perfectly flat but exhibit height variations corresponding to one or two layers of gold NRs ( Figure 10j , k ) , which could be advantageous for surface - enhanced spectroscopy due to the increased accessibility of local hot - spots . However , because of the drying process on a solid substrate ( TEM grid in this case ) , fluctuations in the local NR concentration are expected and the extent of spatial ordering for monodisperse gold NRs is largely limited by the area over which a uniform concentration of NRs can be maintained during the very last stage of deposition . Indeed , this is reflected by the coexistence of smectic and nematic phases on a single TEM grid for gold NRs of various aspect ratios . The nematic phase , characterized by a uniaxial orientational order and lack of positional order , usually forms at less concentrated regions and is more common for NRs of larger aspect ratios ( Figures S47–S49 ) . Interestingly , for gold NRs with an aspect ratio less than 4.0 , bilayered motifs exhibiting approximately centered - rectangular local packing symmetry are routinely observed ( Figures S35 , S36 , S39 , and S45 ) , which has been rarely seen in other NR assemblies . Moreover , the nematic or smectic ordering can be disrupted through continuous bending of neighboring layers , reminiscent of nonequilibrium drying and complex interactions between the NRs ( Figure 10e , f ) . Those wavy gold nanorod assemblies have also been reported by Murphy et al . and referred to as “ swirling clustering ” or “ end - to - end cementing ” . Therefore , theories on the packing of hard rods under equilibrium conditions can not fully explain the observed structural complexity of gold NR assemblies , and development of a rigorous framework taking into account not only entropic contributions but also nanoscale forces including van der Waals attractions , electrostatic repulsions , dipole – dipole interactions , and , in particular , the role of CTAB layers and counterion interaction in self - assembling gold nanorods is needed .  '}, 
{'role': 'assistant', 'content': 'Self - organization of monodisperse nanoscale building blocks into ordered arrays represents a facile and versatile route for metamaterials with rationally modulated properties . In particular , liquid - crystalline - type superstructures or superlattices composed of anisotropic nanocrystals have attracted a fast - growing interest due to the rich phase behaviors resulting from the added complexity of shape anisotropy and the potential for emergent collective properties . Here we explore the intriguing shape - dependent packing symmetry of ordered NR assemblies by slow evaporation of concentrated gold NR aqueous solutions at room temperature . Onsager ’s theory on the assembly behavior of hard @@rodlike## colloids shows that above a critical concentration of rods there is a net gain in entropy on transition from isotropic random packing to a nematic liquid crystalline phase . As shown in Figure 10 , large - area smectic phases form in dense NR solids , where NRs self - organize into close - packed layers and adjacent layers align nearly parallel into ordered superstructures . AFM studies confirm the three - dimensional nature of gold NR assemblies with a typical domain size of ∼10 μm2 ( Figures 10i , S43 ) . AFM height analysis along layers of smectic phases reveals periodic height modulation with a periodicity of 28.0 ± 2.5 nm ( the NR diameter is measured to be 22.0 ± 1.0 nm from TEM images ) , which is consistent with recent experimental studies showing partially interdigitated CTAB bilayers on the gold NR surface ( Figure 10j , k ) . Moreover , the surfaces of gold NR assemblies are not perfectly flat but exhibit height variations corresponding to one or two layers of gold NRs ( Figure 10j , k ) , which could be advantageous for surface - enhanced spectroscopy due to the increased accessibility of local hot - spots . However , because of the drying process on a solid substrate ( TEM grid in this case ) , fluctuations in the local NR concentration are expected and the extent of spatial ordering for monodisperse gold NRs is largely limited by the area over which a uniform concentration of NRs can be maintained during the very last stage of deposition . Indeed , this is reflected by the coexistence of smectic and nematic phases on a single TEM grid for gold NRs of various aspect ratios . The nematic phase , characterized by a uniaxial orientational order and lack of positional order , usually forms at less concentrated regions and is more common for NRs of larger aspect ratios ( Figures S47–S49 ) . Interestingly , for gold NRs with an aspect ratio less than 4.0 , bilayered motifs exhibiting approximately centered - rectangular local packing symmetry are routinely observed ( Figures S35 , S36 , S39 , and S45 ) , which has been rarely seen in other NR assemblies . Moreover , the nematic or smectic ordering can be disrupted through continuous bending of neighboring layers , reminiscent of nonequilibrium drying and complex interactions between the NRs ( Figure 10e , f ) . Those wavy gold nanorod assemblies have also been reported by Murphy et al . and referred to as “ swirling clustering ” or “ end - to - end cementing ” . Therefore , theories on the packing of hard rods under equilibrium conditions can not fully explain the observed structural complexity of gold NR assemblies , and development of a rigorous framework taking into account not only entropic contributions but also nanoscale forces including van der Waals attractions , electrostatic repulsions , dipole – dipole interactions , and , in particular , the role of CTAB layers and counterion interaction in self - assembling gold nanorods is needed .  '}, 

{'role': 'user', 'content': 'In order to monitor the growth of AuNRs by TEM , aliquots were removed from the growth solution every hour , their absorbance spectra were measured , and they were incubated with MUTAB for 2 days to ensure that a maximum surface coverage had occurred . TEM images of these rods after 3 , 7 , and 11 h of growth are shown in Figure 5 ( see Figure S2 , Supporting Information , for the 5 , 9 , and 12 h points ) . Figure 6 details the change in AuNRs dimensions extracted from the TEM images . Specifically , the changes in length ( Figure 6A ) , waist and tip diameter ( Figure 6B ) , and total volume , estimated by assuming a cylindrical structure with diameter averaged between the waist and tips ( Figure 6C ) , are plotted against the growth time . A summary of this information is provided in Table S2 ( Supporting Information ) . After 3 h , AuNRs display the standard cylindrical morphology with the aspect ratio and LSPR peak close to the final values . However , at this point the rods have grown to only about 15 % of their final size based on the estimated volume . The linear increase in volume from that point shown in Figure 6A demonstrates that the reduction to metallic gold occurred at a roughly constant rate , assuming that no new nucleation events are taking place in solution . This steady growth in size is in contrast to the “ popcorn ” model of ascorbic acid - mediated nanorods growth proposed by Edgar and co - workers . This model states that rather than following the traditional assumption that all seed particles grow evenly from the start of the reaction to the end , the seed particles actually lie quiescent for variable amounts of time and then rapidly grow to their final size . This model was supported by the presence of both small and nearly fully grown AuNRs starting from the very beginning of the reaction up until nearly the end , which is clearly not evident in our procedure . However , it may be that due to the reduced growth rate , most seeds that will eventually grow have time to exit the quiescent state within the first few hours of growth while nanorods have only had time to grow ∼15 % of their final size . In other words , the rate at which seed particles switch from the quiescent to the growing state appears to be faster than the rate of AuNRs growth .  '}, 
{'role': 'assistant', 'content': 'In order to monitor the growth of AuNRs by TEM , aliquots were removed from the growth solution every hour , their absorbance spectra were measured , and they were incubated with MUTAB for 2 days to ensure that a maximum surface coverage had occurred . TEM images of these rods after 3 , 7 , and 11 h of growth are shown in Figure 5 ( see Figure S2 , Supporting Information , for the 5 , 9 , and 12 h points ) . Figure 6 details the change in AuNRs dimensions extracted from the TEM images . Specifically , the changes in length ( Figure 6A ) , waist and tip diameter ( Figure 6B ) , and total volume , estimated by assuming a @@cylindrical## structure with diameter averaged between the waist and tips ( Figure 6C ) , are plotted against the growth time . A summary of this information is provided in Table S2 ( Supporting Information ) . After 3 h , AuNRs display the standard @@cylindrical## morphology with the aspect ratio and LSPR peak close to the final values . However , at this point the rods have grown to only about 15 % of their final size based on the estimated volume . The linear increase in volume from that point shown in Figure 6A demonstrates that the reduction to metallic gold occurred at a roughly constant rate , assuming that no new nucleation events are taking place in solution . This steady growth in size is in contrast to the “ popcorn ” model of ascorbic acid - mediated nanorods growth proposed by Edgar and co - workers . This model states that rather than following the traditional assumption that all seed particles grow evenly from the start of the reaction to the end , the seed particles actually lie quiescent for variable amounts of time and then rapidly grow to their final size . This model was supported by the presence of both small and nearly fully grown AuNRs starting from the very beginning of the reaction up until nearly the end , which is clearly not evident in our procedure . However , it may be that due to the reduced growth rate , most seeds that will eventually grow have time to exit the quiescent state within the first few hours of growth while nanorods have only had time to grow ∼15 % of their final size . In other words , the rate at which seed particles switch from the quiescent to the growing state appears to be faster than the rate of AuNRs growth .  '}, 

{'role': 'user', 'content': 'The detailed morphology and component characterizations were presented with TEM images and XRD patterns . Fig . 1 ( a ) gives the TEM image of pure ZnO , and Fig . 1(b ) exhibits the initial Au NRs with an average length / diameter of 15 ± 3 nm/50 ± 3 nm . Fig . 1(c ) shows that the Au@ZnO NRs exhibit a standard core - shell nanostructure . The Au cores with special dumbbell - like morphology is ascribed to the overgrowth of Au NRs , the corresponding growth mechanism had been discussed in our previous work [ 11 ] . The ZnO nanoshells have an average thickness of 52 nm and display a loose form . As observed in Fig . 1(d ) , the annealed Au@ZnO NRs keep uniform core - shell structures , and its Au cores still maintain rod - like morphology . The ZnO nanoshells show dense morphology , suggesting that its crystallinity is highly enhanced .  '}, 
{'role': 'assistant', 'content': 'The detailed morphology and component characterizations were presented with TEM images and XRD patterns . Fig . 1 ( a ) gives the TEM image of pure ZnO , and Fig . 1(b ) exhibits the initial Au NRs with an average length / diameter of 15 ± 3 nm/50 ± 3 nm . Fig . 1(c ) shows that the Au@ZnO NRs exhibit a standard core - shell nanostructure . The Au cores with special @@dumbbell## - @@like## morphology is ascribed to the overgrowth of Au NRs , the corresponding growth mechanism had been discussed in our previous work [ 11 ] . The ZnO nanoshells have an average thickness of 52 nm and display a loose form . As observed in Fig . 1(d ) , the annealed Au@ZnO NRs keep uniform core - shell structures , and its Au cores still maintain @@rod## - @@like## morphology . The ZnO nanoshells show dense morphology , suggesting that its crystallinity is highly enhanced .  '}, 

{'role': 'user', 'content': 'Fig . 4 a corresponds to the TEM images of AuNRs dispersion in the water having length ≈53–60 nm and width ≈20–25 nm ( aspect ratio ≈ 2.5 ) along with some square or rectangular shaped particles ( SI - Fig . 2 ) . MeOH addition resulted in the decrease of inter particle distance among the adjacent NRs and showed parallel alignment to a little extent ( Fig . 4b ) . Formation of small groups of NRs with random or haphazard arrangement ( Fig . 4c ) and a large clump of aggregate ( Fig . 4d ) relative to distantly positioned rods in aqueous media are also seen in MeOH dispersion .  '}]
"""

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='your key')
result= client.chat.completions.create(model='gpt-4-0613',
        messages=prompt, 
        temperature=0,
        max_tokens=256,         
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
pred= result.choices[0].message

2.5 Fine-tuning of GPT-3

In [6]:
train_df = pd.read_csv("your file")
train_df.head()

,prompt,completion
0,"Optoelectronic properties of XIn2S4 ( X = Cd ,...",Optoelectronic properties of @@XIn2S4 ( X = C...
1,Ca - induced surface reconstructions on TiO2(1...,@@Ca## - induced surface reconstructions on @...
2,Magnetostriction and magnetization process of ...,Magnetostriction and magnetization process of...
3,Synthesis of ZnO nanoparticles using surfactan...,Synthesis of @@ZnO## nanoparticles using surf...
4,Effects of solvent and chelating agent on synt...,Effects of solvent and chelating agent on syn...


In [ ]:
prompt_example = """
Optoelectronic properties of XIn2S4 ( X = Cd , Mg ) thiospinels through highly accurate all - electron FP - LAPW method coupled with modified approximations We report the structural , electronic and optical properties of the thiospinels XIn2S4 ( X = Cd , Mg ) , using highly accurate all - electron full potential linearized augmented plane wave plus local orbital method . 
In order to calculate the exchange and correlation energies , the method is coupled with modified techniques such as GGA+U and mBJ - GGA , which yield improved results as compared to the previous studies . 
GGA+SOC approximation is also used for the first time on these compounds to examine the spin orbit coupling effect on the band structure . 
From the analysis of the structural parameters , robust character is predicted for both materials . 
Energy band structures profiles are fairly the same for GGA , GGA+SOC , GGA+U and mBJ - GGA , confirming the indirect and direct band gap nature of CdIn2S4 and MgIn2S4 materials , respectively . We report the trend of band gap results as : ( mBJ - GGA ) > ( GGA+U ) > ( GGA ) > ( GGA+SOC ) . 
Localized regions appearing in the valence bands for CdIn2S4 tend to split up nearly by ≈ 1eV in the case of GGA+SOC . 
Many new physical parameters are reported that can be important for the fabrication of optoelectronic devices . 
Optical spectra namely , dielectric function ( DF ) , refractive index n(ω) , extinction coefficient k(ω) , reflectivity R(ω) , optical conductivity σ(ω) , absorption coefficient α(ω) and electron loss function are discussed . 
Optical ’s absorption edge is noted to be 1.401 and 1.782 for CdIn2S4 and MgIn2S4 , respectively . 
The prominent peaks in the electron energy spectrum situated between 15eV and 23eV for the herein studied materials indicate a transition from metallic to the dielectric character .  ==>\n
"""

completion_example = """ 
Optoelectronic properties of @@XIn2S4 ( X = Cd , Mg )## thiospinels through highly accurate all - electron FP - LAPW method coupled with modified approximations We report the structural , electronic and optical properties of the thiospinels @@XIn2S4 ( X = Cd , Mg )## , using highly accurate all - electron full potential linearized augmented plane wave plus local orbital method . 
In order to calculate the exchange and correlation energies , the method is coupled with modified techniques such as GGA+U and mBJ - GGA , which yield improved results as compared to the previous studies . 
GGA+SOC approximation is also used for the first time on these compounds to examine the spin orbit coupling effect on the band structure . 
From the analysis of the structural parameters , robust character is predicted for both materials . 
Energy band structures profiles are fairly the same for GGA , GGA+SOC , GGA+U and mBJ - GGA , confirming the indirect and direct band gap nature of @@CdIn2S4## and @@MgIn2S4## materials , respectively . We report the trend of band gap results as : ( mBJ - GGA ) > ( GGA+U ) > ( GGA ) > ( GGA+SOC ) . Localized regions appearing in the valence bands for @@CdIn2S4## tend to split up nearly by ≈ 1eV in the case of GGA+SOC . 
Many new physical parameters are reported that can be important for the fabrication of optoelectronic devices . 
Optical spectra namely , dielectric function ( DF ) , refractive index n(ω) , extinction coefficient k(ω) , reflectivity R(ω) , optical conductivity σ(ω) , absorption coefficient α(ω) and electron loss function are discussed . 
Optical ’s absorption edge is noted to be 1.401 and 1.782 for @@CdIn2S4## and @@MgIn2S4## , respectively . 
The prominent peaks in the electron energy spectrum situated between 15eV and 23eV for the herein studied materials indicate a transition from metallic to the dielectric character .  \n\n###\n\n
"""

In [ ]:
!openai tools fine_tunes.prepare_data -f /dataset/train_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/val_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/test_df.jsonl -q

In [ ]:
!openai api fine_tunes.create -t "train.jsonl" -v "valid.jsonl" -m "davinci" --suffix f"YOUR ALIAS" --batch_size 1 --n_epochs 4 --learning_rate_multiplier 0.01 --prompt_loss_weight 0.01

In [ ]:
openai.organization =""
openai.api_key =""
your_model = ""
prompt = "Graphene encapsulated Fe3O4 nanorods assembled into a mesoporous hybrid composite used as a high - performance lithium - ion battery anode material The discovery of new anode materials and engineering their fine structures are the core elements in the development of new - generation lithium ion batteries ( LIBs ) . To this end , we herein report a novel nanostructured composite consisting of approximately 75 % Fe3O4 nanorods and 25 % reduced graphene oxide ( rGO ) . Microscopy and spectroscopy analyses have identified that the Fe3O4 nanorods are wrapped ( or encapsulated ) by the rGO nanosheets via covalent bonding , which further self - assemble into a mesoporous hybrid composite networked by the graphene matrix . The composite has an average pore size around 20 nm and exhibits a high surface area of 152 m2 g−1 , which is 76 times as high as that of conventional Fe3O4 powder . We have used the composite as an LIB anode material to fabricate coin - type prototype cells with lithium as the cathode . Systematic half - cell testing evaluations show that the electrochemical performance of the present composite material is amongst the best of the transition metal - oxide based LIB anode materials . The performances are characterized by a high reversible capacity of 1053 mA h g−1 subjected to 250 charge – discharge cycles at 500 mA g−1 and an excellent rate capability with the deliverable energy of 788 – 541 mA h g−1 upon the application of high current densities of 1000 – 5000 mA g−1 . Overall , we have demonstrated that Fe3O4 nanorod – rGO hybrid composite is an interesting and promising material for the fabrication of LIB anodes .  ==>\n"
result = openai.Completion.create(model=your_model,        
            prompt=prompt,
            temperature=0,
            max_tokens=256,
            best_of=1,
            top_p=1,
            stop=["\n\n###\n\n"]
            )
print(result)
# " @@Graphene## encapsulated @@Fe3O4## nanorods assembled into a mesoporous hybrid composite used as a high - performance lithium - ion battery anode material The discovery of new anode materials and engineering their fine structures are the core elements in the development of new - generation lithium ion batteries ( LIBs ) . To this end , we herein report a novel nanostructured composite consisting of approximately 75 % @@Fe3O4## nanorods and 25 % @@reduced graphene oxide## ( @@rGO## ) . Microscopy and spectroscopy analyses have identified that the @@Fe3O4## nanorods are wrapped ( or encapsulated ) by the @@rGO## nanosheets via covalent bonding , which further self - assemble into a mesoporous hybrid composite networked by the @@graphene## matrix . The composite has an average pore size around 20 nm and exhibits a high surface area of 152 m2 g−1 , which is 76 times as high as that of conventional @@Fe3O4## powder . We have used the composite as an LIB anode material to fabricate coin - type prototype cells with @@lithium## as the cathode . Systematic half - cell testing evaluations show that the electrochemical performance of the present composite material is amongst the best of the @@transition metal - oxide## based LIB anode materials . The performances are characterized by a high reversible capacity of 1053 mA h g−1 subjected to 250 charge – discharge cycles at 500 mA g−1 and an excellent rate capability with the deliverable energy of 788 – 541 mA h g−1 upon the application of high current densities of 1000 – 5000 mA g−1 . Overall , we have demonstrated that @@Fe3O4## nanorod – @@rGO## hybrid composite is an interesting and promising material for the fabrication of LIB anodes .  \n\n###\n\n"

3. Extractive Q&A

3.1 Zero-Shot Learning with GPT-3.5

In [ ]:
prompt = """ 
Context: The blended slurry was then cast onto a clean current collector (Al foil for the cathode and Cu foil for the anode) and dried at 90 °C under vacuum overnight.
Question: What is the cathode?
"""

3.2 Task-Informed Zero-Shot Learning with GPT-3.5

In [ ]:
prompt = """ 
The task is to extract the tokens corresponding to the answer of question from the given context.
Context: The blended slurry was then cast onto a clean current collector (Al foil for the cathode and Cu foil for the anode) and dried at 90 °C under vacuum overnight.
Question: What is the cathode?
"""

In [ ]:
import openai
num_tokens = 256
result= openai.Completion.create(model='text-davinci-003', 
        prompt=prompt, 
        temperature=0,
        max_tokens=num_tokens, logprobs=2
        )
top_logprobs = result["choices"][0]["logprobs"]["top_logprobs"][0]
token_probs = {
    token: np.exp(logp) 
for token, logp in top_logprobs.items()
}
result['choices'][0]['text'] 

3.3 Fine-tuning of GPT-3

In [9]:
train_df= pd.read_excel('your file')
train_df.head()

,prompt,completion
0,prompt: The blended slurry was then cast onto ...,Al foil
1,prompt: The blended slurry was then cast onto ...,Copper (Cu) foil
2,"prompt: In conclusion, the SiC/RGO nanocomposi...",SiC/RGO nanocomposite
3,prompt: The calculated diffusion coefficient o...,NV NSs@ACC
4,prompt: Li-ion batteries (LIBs) are expected t...,"Ni-rich layered oxides, LiNixCoyAlzO2 (NCA) an..."


In [ ]:
prompt_example = """ 
Context: The blended slurry was then cast onto a clean current collector (Al foil for the cathode and Cu foil for the anode) and dried at 90 °C under vacuum overnight.
Question: What is the cathode?
"""
completion_example = "Al foil"

In [ ]:
!openai tools fine_tunes.prepare_data -f /dataset/train_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/val_df.jsonl -q
!openai tools fine_tunes.prepare_data -f /dataset/test_df.jsonl -q

In [ ]:
!openai api fine_tunes.create -t "train.jsonl" -v "valid.jsonl" -m "davinci" --suffix f"YOUR ALIAS" --batch_size 1 --n_epochs 4 --learning_rate_multiplier 0.01 --prompt_loss_weight 0.01

In [ ]:
openai.organization =""
openai.api_key =""
your_model = ""
prompt = "High energy density lithium ion batteries have been applied as the main energy storage technology in portable devices and green transportation. LiCoO2, a layered oxide cathode in lithium ion batteries, has obtained unprecedented success since its commercialization in the 1990s. LiNiO2, isostructural to LiCoO2, can deliver a higher capacity within a relatively low upper cutoff voltage. However, its practical application has been hindered due to the inherent structural instability. Thus, cobalt, as an essential element, has been incorporated into the LiNiO2 system to form LiNi1−x−yMnyCoxO2 (NMC) and LiNi1−x−yCoxAlyO2 (NCA) materials to release the strong magnetic moment and ultimately stabilize the structure. At present, researchers are pursuing the theoretical capacity of NMCs either by elevating the high cutoff voltage or increasing the nickel concentration. However, these strategies have inevitably provoked some of the degradation pathways of these materials, including transition metal reduction/dissolution, surface reconstruction, gas evolution, electrolyte oxidation/decomposition, and chemomechanical breakdown. Additionally, most of these materials are synthesized in the polycrystalline form and have heterogeneous chemistry, resulting in high heterogeneous degradation patterns."
result = openai.Completion.create(model=your_model,        
            prompt=prompt,
            temperature=0,
            max_tokens=256,
            best_of=1,
            top_p=1,
            stop=["\n\n###\n\n"]
            )
print(result)
# LiCoO2
